In [1]:
# Core
import numpy as np
import pandas as pd

# Tensorflow / ML
# import tensorflow as tf

# Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl

# Local imports
from asteroid_data import load_ast_elt
from candidate_element import orbital_element_batch, perturb_elts, random_elts
from ztf_data import load_ztf_nearest_ast, load_ztf_batch, elt_hash
from ztf_data import make_ztf_batch, calc_hit_freq

In [2]:
# Set plot style variables
mpl.rcParams['figure.figsize'] = [16.0, 10.0]
mpl.rcParams['font.size'] = 16

## Batches of Orbital Elements: Unperturbed, Perturbed, Random

In [3]:
# Load all the asteroid elements
ast_elt = load_ast_elt()

In [4]:
ast_elt

,Num,Name,epoch_mjd,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri
Num,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.34,0.12,JPL 46,1.501306,1683.145749,0.003733,4.036516,4.187424,2.686118,-361.745873
2,2,Pallas,58600.0,2.772466,0.230337,0.608007,3.020817,5.411373,1.041946,4.13,0.11,JPL 35,1.490912,1686.155979,0.003726,3.190951,3.639917,2.149005,-279.616804
3,3,Juno,58600.0,2.669150,0.256942,0.226699,2.964490,4.330836,0.609557,5.33,0.32,JPL 108,0.996719,1592.787270,0.003945,1.621697,2.008860,1.012141,-154.522558
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.20,0.32,JPL 34,-4.436417,1325.432768,0.004740,6.115656,0.006132,4.442550,-352.940421
5,5,Astraea,58600.0,2.574249,0.191095,0.093672,2.470978,6.260280,4.928221,6.85,0.15,JPL 108,-1.738676,1508.600442,0.004165,1.093108,0.709396,2.448072,325.328481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255499,1255499,2019 QG,58600.0,0.822197,0.237862,0.220677,5.066979,3.770460,0.503214,21.55,0.15,JPL 1,0.807024,272.309165,0.023074,-3.225717,-2.921908,-3.728932,-21.808984
1255501,1255501,2019 QL,58600.0,2.722045,0.530676,0.113833,4.741919,2.351059,5.297173,19.21,0.15,JPL 1,-2.082964,1640.368337,0.003830,-0.176219,-1.273172,0.809793,257.420824
1255502,1255502,2019 QQ,58600.0,1.053137,0.389091,0.172121,5.648270,2.028352,3.266522,25.31,0.15,JPL 1,-3.081905,394.753268,0.015917,-1.623227,-1.688469,1.393436,189.527723


In [5]:
# Load ztf nearest asteroid data
ztf_ast = load_ztf_nearest_ast()

In [6]:
ztf_ast

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz,mag_app,asteroid_prob,nearest_ast_num,nearest_ast_dist,ast_ra,ast_dec,ast_ux,ast_uy,ast_uz
0,b'ZTF18acebhfp',676397301515010013,14490,58430.397303,41.357345,58.879488,0.387942,0.653853,0.649598,18.946699,0.865682,1208789,0.005029,41.396388,58.592038,0.390926,0.655662,0.645976
1,b'ZTF18abodmwk',596403415715010014,5831,58350.403414,30.969721,65.305308,0.358224,0.558644,0.748059,19.010401,0.855504,1227812,0.024428,33.729101,64.536183,0.357570,0.578168,0.733393
2,b'ZTF18abodmwk',626428345715010011,10614,58380.428345,30.969705,65.305294,0.358224,0.558644,0.748059,18.935900,0.855504,1169677,0.015510,29.207596,64.817653,0.371401,0.550472,0.747691
3,b'ZTF18abodmwk',630507595715015045,11250,58384.507593,30.969940,65.305305,0.358223,0.558645,0.748059,19.260401,0.855504,1251951,0.012386,30.227911,65.945543,0.352183,0.551505,0.756181
4,b'ZTF18abodmwk',618384965715010022,9040,58372.384965,30.969643,65.305179,0.358226,0.558644,0.748058,19.220200,0.855504,1246591,0.025343,34.169666,64.771024,0.352659,0.579473,0.734740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5697957,b'ZTF20aareruw',1151532523515015015,97109,58905.532523,253.007910,55.485537,-0.165587,-0.169403,0.971537,19.192400,0.608023,1102168,0.036944,253.707834,53.408139,-0.167230,-0.205583,0.964246
5697958,b'ZTF20aarerwx',1151533002615015009,97110,58905.533009,232.886408,53.509617,-0.358833,-0.115301,0.926253,19.687099,0.559474,1028157,0.084402,224.967815,54.919912,-0.406617,-0.047125,0.912382
5697959,b'ZTF20aarerww',1151533002115010003,97110,58905.533009,236.167899,54.618457,-0.322375,-0.116973,0.939357,19.957001,0.392662,539940,0.052254,240.693936,56.155104,-0.272611,-0.115218,0.955200
5697960,b'ZTF20aarervr',1151526063515015015,97098,58905.526065,286.235286,33.876902,0.232120,-0.509626,0.828494,19.049299,0.517241,1246304,0.014054,285.998189,34.657915,0.226704,-0.499251,0.836274


In [7]:
# Asteroid numbers and hit counts
ast_num, hit_count = calc_hit_freq(ztf=ztf_ast, thresh_sec=2.0)

# Sort the hit counts in descending order and find the top batch_size
idx = np.argsort(hit_count)[::-1]

# Extract the asteroid number and hit count for this batch
ast_num_best = ast_num[idx]
hit_count_best = hit_count[idx]

In [8]:
# Set batch size
# batch_size = 1000
batch_size = 64

# Batch of unperturbed elements
elts = orbital_element_batch(ast_nums=ast_num_best[0:batch_size])

# Add hit_count column
elts['hit_count'] = hit_count_best[0:batch_size]

elts

,element_id,a,e,inc,Omega,omega,f,epoch,hit_count
0,51921,2.669306,0.217361,0.499554,4.699703,2.450796,-1.133491,58600.0,194
1,59244,2.634727,0.262503,0.465045,5.738297,1.766995,-1.601363,58600.0,190
2,15786,1.883227,0.047655,0.392360,6.134689,0.804823,-1.246069,58600.0,189
3,3904,2.556387,0.098279,0.261542,5.450163,2.202423,-1.357345,58600.0,188
4,142999,2.619944,0.191376,0.514017,0.238022,0.946463,-1.299301,58600.0,186
...,...,...,...,...,...,...,...,...,...
59,11952,2.219650,0.086091,0.117967,0.042442,2.904823,-3.016580,58600.0,148
60,134815,2.612770,0.140831,0.513923,0.272689,0.645552,-0.957836,58600.0,148
61,27860,2.619406,0.096185,0.200633,5.541399,3.266046,3.948770,58600.0,148
62,85937,2.342292,0.197267,0.439063,5.279693,3.210025,3.947687,58600.0,148


In [9]:
np.mean(elts.hit_count)

161.453125

In [10]:
# Alias elts to elts_ast for naming consistency with elts_pert and elts_rand
elts_ast = elts.copy()

In [11]:
# Perturb orbital elements
sigma_a = 0.0 
sigma_e = 0.0 
sigma_f_deg = 0.1
sigma_Omega_deg = 0.0
sigma_omega_deg = 0.0
mask_pert = None
random_seed = 42

elts_pert = perturb_elts(elts, sigma_a=sigma_a, sigma_e=sigma_e, sigma_f_deg=sigma_f_deg, 
                         sigma_Omega_deg=sigma_Omega_deg, sigma_omega_deg=sigma_omega_deg,
                         mask_pert=mask_pert, random_seed=random_seed)

In [12]:
# Batch of random elements
elts_rand = random_elts(element_id_start=0, size=batch_size, random_seed=42)

## Batches of ZTF Data vs. Elements

In [13]:
# Arguments to make_ztf_batch
# thresh_deg = 10.0
thresh_deg = 1.0
near_ast = False

In [14]:
# Load unperturbed element batch
ztf_elt_ast = load_ztf_batch(elts=elts_ast, thresh_deg=thresh_deg, near_ast=near_ast)

In [15]:
# Review 
ztf_elt_ast

,ztf_id,element_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,...,elt_ux,elt_uy,elt_uz,elt_r,s,dist_sec,z,v,score,is_hit
0,53851,733,b'ZTF18abnothj',594197584815010004,5501,58348.197581,266.229165,-13.513802,-0.063945,-0.983101,...,-0.057300,-0.982042,0.179751,2.234078,0.010624,2191.371398,0.999944,0.370539,0.009739,False
1,73604,733,b'ZTF18ablwzmb',594197584815015003,5501,58348.197581,265.761024,-13.509148,-0.071871,-0.982578,...,-0.057300,-0.982042,0.179751,2.234078,0.016809,3467.103003,0.999859,0.927533,0.000009,False
2,82343,733,b'ZTF18abiydvm',635193253015015018,12089,58389.193252,270.331454,-11.244934,0.005674,-0.977422,...,0.000918,-0.977996,0.208622,2.703478,0.005450,1124.142942,0.999985,0.097510,0.295572,False
3,257221,733,b'ZTF18acakcqg',931471223715015007,39920,58685.471227,29.693832,42.180412,0.643725,0.603886,...,0.639004,0.610779,0.467571,2.175851,0.008712,1797.042210,0.999962,0.249184,0.044390,False
4,327000,733,b'ZTF18achmdmw',937465970615015011,40837,58691.465972,33.104905,44.059131,0.601970,0.636719,...,0.606278,0.637608,0.475272,2.114865,0.007949,1639.537152,0.999968,0.207418,0.074820,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90205,5650588,324582,b'ZTF20aaqvhld',1150176701515015008,96618,58904.176701,44.164238,29.650540,0.623416,0.752309,...,0.627640,0.750696,0.206212,2.981799,0.008187,1688.636853,0.999966,0.220027,0.063910,False
90206,5650589,324582,b'ZTF20aaqvhld',1150176245715015005,96617,58904.176250,44.164062,29.650536,0.623417,0.752307,...,0.627641,0.750695,0.206213,2.981793,0.008187,1688.600639,0.999966,0.220018,0.063917,False
90207,5650665,324582,b'ZTF20aaqvhll',1150176245815015010,96617,58904.176250,44.368640,28.490480,0.628284,0.753618,...,0.627641,0.750695,0.206213,2.981793,0.013370,2757.856469,0.999911,0.586871,0.000652,False
90208,5650697,324582,b'ZTF20aaqvhmb',1150176246015015005,96617,58904.176250,43.296207,29.505908,0.633424,0.743491,...,0.627641,0.750695,0.206213,2.981793,0.012388,2555.279465,0.999923,0.503822,0.001841,False


In [16]:
np.sum(ztf_elt_ast.is_hit)

10331

In [17]:
# Load perturbed element batch
ztf_elt_pert = load_ztf_batch(elts=elts_pert, thresh_deg=thresh_deg, near_ast=near_ast)

In [18]:
# Review 
ztf_elt_pert

,ztf_id,element_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,...,elt_ux,elt_uy,elt_uz,elt_r,s,dist_sec,z,v,score,is_hit
0,53851,733,b'ZTF18abnothj',594197584815010004,5501,58348.197581,266.229165,-13.513802,-0.063945,-0.983101,...,-0.057188,-0.982035,0.179826,2.233936,0.010753,2217.904668,0.999942,0.379566,0.008699,False
1,73604,733,b'ZTF18ablwzmb',594197584815015003,5501,58348.197581,265.761024,-13.509148,-0.071871,-0.982578,...,-0.057188,-0.982035,0.179826,2.233936,0.016943,3494.885066,0.999856,0.942457,0.000008,False
2,82343,733,b'ZTF18abiydvm',635193253015015018,12089,58389.193252,270.331454,-11.244934,0.005674,-0.977422,...,0.001009,-0.977983,0.208684,2.703306,0.005340,1101.510825,0.999986,0.093623,0.310286,False
3,257221,733,b'ZTF18acakcqg',931471223715015007,39920,58685.471227,29.693832,42.180412,0.643725,0.603886,...,0.638907,0.610928,0.467509,2.175998,0.008901,1835.974314,0.999960,0.260098,0.038729,False
4,327000,733,b'ZTF18achmdmw',937465970615015011,40837,58691.465972,33.104905,44.059131,0.601970,0.636719,...,0.606174,0.637757,0.475204,2.115016,0.007968,1643.596206,0.999968,0.208446,0.073864,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90544,5650589,324582,b'ZTF20aaqvhld',1150176245715015005,96617,58904.176250,44.164062,29.650536,0.623417,0.752307,...,0.626528,0.751658,0.206089,2.980290,0.007641,1576.070983,0.999971,0.191671,0.091097,False
90545,5650665,324582,b'ZTF20aaqvhll',1150176245815015010,96617,58904.176250,44.368640,28.490480,0.628284,0.753618,...,0.626528,0.751658,0.206089,2.980290,0.013172,2717.023101,0.999913,0.569622,0.000809,False
90546,5650694,324582,b'ZTF19aaevkwf',1150176701515015007,96618,58904.176701,45.057609,29.656244,0.613864,0.761162,...,0.626527,0.751660,0.206088,2.980296,0.016152,3331.622428,0.999870,0.856462,0.000022,False
90547,5650697,324582,b'ZTF20aaqvhmb',1150176246015015005,96617,58904.176250,43.296207,29.505908,0.633424,0.743491,...,0.626528,0.751658,0.206089,2.980290,0.013582,2801.422169,0.999908,0.605559,0.000516,False


In [20]:
# Load random element batch
ztf_elt_rand = load_ztf_batch(elts=elts_rand, thresh_deg=thresh_deg, near_ast=near_ast)

In [21]:
ztf_elt_rand

,ztf_id,element_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,...,elt_ux,elt_uy,elt_uz,elt_r,s,dist_sec,z,v,score,is_hit
0,96823,0,b'ZTF18abvfbei',708277241115015000,18391,58462.277245,15.633115,16.050475,0.925468,0.347581,...,0.923639,0.350978,0.153963,2.107494,0.005081,1048.005013,0.999987,0.084748,0.346688,False
1,96824,0,b'ZTF18abvfbei',720160041115010000,19954,58474.160046,15.633036,16.050428,0.925468,0.347580,...,0.923696,0.355835,0.142011,2.247321,0.012084,2492.496946,0.999927,0.479369,0.002499,False
2,96825,0,b'ZTF18abvfbei',703201851115015000,18190,58457.201852,15.633098,16.050466,0.925468,0.347581,...,0.921753,0.353649,0.159071,2.052863,0.011020,2272.970994,0.999939,0.398648,0.006853,False
3,96827,0,b'ZTF18abvfbei',717122091115010000,19362,58471.122095,15.633029,16.050405,0.925468,0.347579,...,0.924249,0.353170,0.145033,2.210267,0.008023,1654.769593,0.999968,0.211290,0.071285,False
4,96828,0,b'ZTF18abvfbei',717162841115010001,19411,58471.162847,15.633006,16.050428,0.925468,0.347579,...,0.924245,0.353196,0.144993,2.210760,0.008069,1664.388261,0.999967,0.213753,0.069123,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67956,5650542,63,b'ZTF20aaqvhhw',1150174895215015003,96614,58904.174896,55.836422,8.024680,0.556059,0.807255,...,0.549457,0.811172,-0.200245,2.640927,0.008046,1659.654697,0.999968,0.212539,0.070180,False
67957,5651189,63,b'ZTF20aaqvkuw',1150186701015015008,96637,58904.186701,55.877823,8.132705,0.555318,0.808165,...,0.549412,0.811205,-0.200231,2.641083,0.007774,1603.459986,0.999970,0.198390,0.083758,False
67958,5651330,63,b'ZTF20aaqvlet',1150186701115015002,96637,58904.186701,57.021720,8.583610,0.538224,0.820403,...,0.549412,0.811205,-0.200231,2.641083,0.016182,3337.905000,0.999869,0.859695,0.000022,False
67959,5651501,63,b'ZTF20aaqvkti',1150186700915015010,96637,58904.186701,56.117308,8.941580,0.550719,0.814245,...,0.549412,0.811205,-0.200231,2.641083,0.016945,3495.178854,0.999856,0.942615,0.000008,False


## Group Results by Element

In [66]:
def score_by_elt(ztf_elt):
    """Score the ztf observations by element"""
    # Add log(v) column
    ztf_elt['log_v'] = np.log(ztf_elt.v)

    # Group by element_id
    elt_score = ztf_elt['log_v'].groupby(ztf_elt.element_id).agg(['count', 'mean', 'std'])

    # Rename columns
    col_name_tbl = {
        'count': 'n_obs',
        'mean': 'log_v_mean',
        'std': 'log_v_std',
    }
    elt_score.rename(columns=col_name_tbl, inplace=True)
    # Add t-score column
    elt_score['t'] = elt_score.log_v_mean * np.sqrt(elt_score.n_obs)

    # Return score grouped by element
    return elt_score

In [69]:
# Score the three batches by element
elt_score_ast = score_by_elt(ztf_elt_ast)
elt_score_pert = score_by_elt(ztf_elt_pert)
elt_score_rand = score_by_elt(ztf_elt_rand)

In [68]:
elt_score_ast

,n_obs,log_v_mean,log_v_std,t
element_id,,,,
733,1424,-3.138430,5.440304,-118.431572
1476,936,-4.066002,6.212529,-124.395746
1803,1117,-3.632913,5.699910,-121.417573
2015,1087,-3.758059,6.234917,-123.902002
2294,1035,-3.770954,6.128471,-121.316923
...,...,...,...,...
203722,3127,-1.869182,3.620183,-104.523857
253246,943,-3.444775,5.696939,-105.783190
306781,1726,-2.436412,4.674685,-101.221137


In [100]:
def score_batches(elt_score_tbl: dict):
    """
    Summarize the score over a batch
    INPUTS:
        elt_score_tbl: Dict. key = batch_name, value= elt_score DataFrame
    """
    # Initialize batch_score frame
    batch_score = pd.DataFrame(columns=['t_mean', 't_std']) 
    # Iterate through scores in the table
    for batch_name, elt_score in elt_score_tbl.items():
        t_mean, t_std = elt_score['t'].agg(['mean', 'std'])
        batch_score.loc[batch_name] = [t_mean, t_std]
        
    return batch_score
    

In [79]:
elt_score_tbl = {
    'ast': elt_score_ast,
    'pert': elt_score_pert,
    ''
}

-111.87145726422074

In [97]:
elt_score = elt_score_ast.copy()
batch_name = 'ast'

batch_score = pd.DataFrame(columns=['t_mean', 't_std'])
t_mean, t_std = elt_score['t'].agg(['mean', 'std'])
batch_score.loc[batch_name] = [t_mean, t_std]

# batch_score.loc[batch_name, 't_mean'] = t_mean
# batch_score['t_std'] = t_std

In [98]:
batch_score

,t_mean,t_std
ast,-111.871457,8.745849


## Plot Distribution of v

In [24]:
def report_v(v: np.ndarray, elt_name: str):
    """Summary statistics for v and log(v)"""

    # Summary statistics for v
    # E[V] = 0.5, Var[V] = 1/12
    v_mean = np.mean(v)
    v_std = np.std(v)
    v_min = np.min(v)
    v_max = np.max(v)
    
    # Number of observations
    n = v.size

    # Summary statistics for log(v)
    # E[log(V)] = -1.0, Var[log(V)] = 1.0
    log_v_mean = np.mean(np.log(v))
    log_v_std = np.std(np.log(v))
    # t-score; theoretical standard deviation for 
    # log_v_t = (-log_v_mean - 1.0) / (log_v_std * np.sqrt(n))
    log_v_t = (-log_v_mean - 1.0) * np.sqrt(n)
    
    # Report
    print(f'v = (1-z) / (1-z_thresh) for {elt_name} asteroid elements:')
    print(f'Mean: {v_mean:10.6f}')
    print(f'Std : {v_std:10.6f}')
    # print(f'Min : {v_min:10.6f}')
    # print(f'Max : {v_max:10.6f}')
    print(f'\nlog(v):')
    print(f'Mean: {log_v_mean:10.6f}')
    print(f'Std : {log_v_std:10.6f}')
    print(f't   : {log_v_t:10.6f}')

In [27]:
# Report unperturbed elements
report_v(ztf_elt_ast.v, elt_name='unperturbed')

v = (1-z) / (1-z_thresh) for unperturbed asteroid elements:
Mean:   0.438113
Std :   0.314616

log(v):
Mean:  -2.856457
Std :   5.196363
t   : 557.586408


In [25]:
# Report unperturbed elements
report_v(ztf_elt_pert.v, elt_name='perturbed')

v = (1-z) / (1-z_thresh) for perturbed asteroid elements:
Mean:   0.444730
Std :   0.311974

log(v):
Mean:  -1.462553
Std :   1.743044
t   : 139.188586


In [28]:
# Report random elements
report_v(ztf_elt_rand.v, elt_name='random')

v = (1-z) / (1-z_thresh) for random asteroid elements:
Mean:   0.494964
Std :   0.288527

log(v):
Mean:  -1.013261
Std :   1.001889
t   :   3.457109


In [ ]:
# Plot density of v
fig, ax = plt.subplots()
ax.set_title('Score Histogram: v')
ax.set_xlabel('v')
ax.set_ylabel('Density')
n, bins, patches = ax.hist(x=v, bins=101, density=True, cumulative=False, color='blue')
ax.axhline(1.0, color='red')
ax.set_xlim([0.0, 1.0])
# ax.legend()
ax.grid()
# fig.savefig('../figs/elts/elt_hist_a.png', bbox_inches='tight')
plt.show()